In [21]:
import jax 
import jax.numpy as jnp
from jax import random, scipy
from typing import NamedTuple
import matplotlib.pyplot as plt
guassian_pdf = jax.scipy.stats.multivariate_normal.pdf 

## Utils

In [108]:
class Hyper(NamedTuple):
    #Hyperparameters with default settings..
    p = 0.75 # partipation rate
    L = 2 # length
    b = 20 #batch size
    N = 10_000 #total number of examples
    n = 100 # number of agents 
    d = 20 # dimension of the experiment 
    gamma = 10e-2 #hyper parameter
    alpha = 10e-2 # the total things
    epochs = 500
    repeats = 1
    seed = 42
    
def _outer(a,b):
    return jnp.outer(a,b)
outer = jax.vmap(_outer, (0,0),0)
outer_clients = jax.vmap(outer,(0,0),0)


def _get_prob(a, mean, sigma):
    return guassian_pdf(a,mean,sigma)
_get_prob_1 = jax.vmap(_get_prob, (0,None,None),0)
get_prob = jax.vmap(_get_prob_1,(None,0,None),1)    


@jax.jit
def client_run(data, means, SS_1, SS_2, vss_1, vss_2, sigma):
    def _get_ss2(prob_vec,data):
        prob_vec = jnp.expand_dims(prob_vec, axis=1) # (n,1)
        return jnp.mean(prob_vec*data ,axis=1) #(n,d) -> (d)
    get_ss2 = jax.vmap(_get_ss2, (0,None),0) #(L,d)
    
    prob = get_prob(data,means,sigma) #(n,L)
    sum_prob = jnp.sum(prob, axis=1)  #(n)
    sum_prob = jnp.expand_dims(sum_prob,1) #(n,1)
    raw_ss_1 = prob/sum_prob #(n,L)
    ss_1 = jnp.mean(raw_ss_1, axis=0) #(L)
    ss_2 = get_ss2(raw_ss_1.T, data) #(L,)
    delta1 = SS_1-ss_1-vss_1
    delta2 = SS_2-ss_2-vss_2
    return delta1, delta2

def generate_data(key, hyper:Hyper):
    #beaware of for loop here
    key, *subkey = random.split(key, num=12)
    subkey = (i for i in subkey)
    means = random.normal(next(subkey), shape=(hyper.L,hyper.d))
    sigma = jnp.cov(random.normal(next(subkey), shape=(hyper.d,hyper.d))@random.normal(next(subkey),shape=(hyper.d,100_000)))
    source_data = [random.multivariate_normal(next(subkey), means[i], sigma,shape=(hyper.N,)) for i in range(hyper.L)]
    logits  = random.dirichlet(next(subkey),1.5* jnp.array([1.0 for _ in range(hyper.L)]))
    mixture_weights = jax.nn.softmax(logits)
    cuts = random.categorical(next(subkey), logits, shape=(hyper.N,))
    raw_data = jnp.vstack([dt[cuts==idx] for idx,dt in enumerate(source_data)])
    clients_data = jnp.array(jnp.split(random.permutation(next(subkey),raw_data), hyper.n))
    return key,means,sigma,mixture_weights,clients_data 


def compute_likelyhood(data, means, mixture_weights):
    #Need to use the prob_ffunction
    def single_likelyhoo(data,means):
        scipy.norm
    pass

## Main

In [ ]:
def main():
    hyper = Hyper()
    key = random.PRNGKey(hyper.seed)
    # Generate the data
    key, t_means,t_sigma,t_mixture_weights, clients_data =  generate_data(key, hyper)
    yTy = outer_clients(clients_data,clients_data)
    yTy = jnp.mean(yTy,axis=(0,1))
    #Initialize the variables
    key, means, sigma, mixture_weights, _ =  generate_data(key, hyper)
    Msg1 = [1/hyper.n * jnp.ones(shape=(hyper.L,)) for i in range(hyper.n)] #(L)
    Msg2 = [1/hyper.n * jnp.ones(shape=(hyper.L,hyper.d)) for i in range(hyper.n)] #(L,d)
    Vss1 = [1/hyper.n * jnp.ones(shape=(hyper.L,)) for i in range(hyper.n)] #(L)
    Vss2 = [1/hyper.n * jnp.ones(shape=(hyper.L,hyper.d)) for i in range(hyper.n)] #(L,d)
    for i in range(hyper.epochs):
        key, *subkey = random.split(key,num=hyper.n+2)
        subkey = (_ for _ in subkey)  
        indicator = random.bernoulli(next(subkey),hyper.p,shape=(hyper.n,))
        participate = jnp.nonzero(indicator)
        delta1, delta2 = [],[]
        for client in participate:
            batch_data = clients_data[i][random.permutation(next(subkey),20)[:hyper.b]]
            # to be fized
            d1, d2 = client_run(batch_data, means, Vss1[i],Vss2[i],sigma)
            delta1.append(d1)
            delta2.append(d2)
            
            break    
        break
            
            
main()

**One Epoc**

In [76]:
key, subkey = random.split(key)
#sample clients:
participate = random.bernoulli(subkey,hyper.p,shape=(hyper.n,))
p_clients = jnp.nonzero(participate)[0]
VR = [None for i in range(hyper.n)]
SS_1 = [None for i in range(hyper.n)]
SS_2 = [None for i in range(hyper.n)]
for i in p_clients:
    

0
1
4
5
6
7
8
9
11
12
13
15
16
17
21
22
24
25
27
28
29
32
33
34
35
36
39
40
41
42
45
46
47
48
49
50
51
52
53
54
55
57
58
59
60
61
62
63
64
65
67
69
70
71
73
74
75
76
77
79
80
82
83
84
85
86
88
89
90
91
92
93
95
97
98
99


In [ ]:
print(type(clients_data))
print(clients_data.shape)

In [ ]:
cat = random.categorical(next(subkey), mixture)


In [ ]:
cuts = random.multinomial

In [ ]:
cuts = np.random.multinomial(N,pvals=mixture, size=None)

In [ ]:
means = random.normal(subkey[0], shape=(L,D))
means = np.random.normal(size=(L,d))
data = [np.random.normal(size=(N,d),loc=means[i]) for i in range(L)]
mixture = np.random.dirichlet(alpha=(2,2),size=None)
cuts = np.random.multinomial(N,pvals=mixture, size=None)

In [ ]:
means = np.random.normal(size=(L,d))
data = [np.random.normal(size=(N,d),loc=means[i]) for i in range(L)]
mixture = np.random.dirichlet(alpha=(2,2),size=None)
cuts = np.random.multinomial(N,pvals=mixture, size=None)
raw_data = np.vstack([data[idx][:i] for idx, i in enumerate(cuts)])
raw_data = raw_data[np.random.permutation(N)]
clients = np.array(np.split(raw_data,n))